In [78]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [79]:
customers= pd.read_csv('/content/Customers.csv')

In [80]:
products= pd.read_csv('/content/Products.csv')

In [81]:
transactions= pd.read_csv('/content/Transactions.csv')

In [82]:
data = transactions.merge(customers, on='CustomerID',how='left').merge(products, on='ProductID',how='left')

In [83]:
customer_transactions = data.groupby(['CustomerID', 'ProductID'])['Quantity'].sum().unstack(fill_value=0)

In [84]:
similarity_matrix = cosine_similarity(customer_transactions)

In [85]:
similarity_df = pd.DataFrame(similarity_matrix, index=customer_transactions.index, columns=customer_transactions.index)

In [86]:
#recommended top 3 similar customer
def recommend_similar_customers(input_customer_id, similarity_df):
    # Validate if the customer ID exists
    if input_customer_id not in similarity_df.index:
        return f"CustomerID {input_customer_id} not found in the dataset. Please check the input."

    # Compute top 3 similar customers excluding the customer itself
    similar_customers = similarity_df.loc[input_customer_id].sort_values(ascending=False)[1:4]  # Exclude self
    recommendations = [
        {"CustomerID": cust_id, "SimilarityScore": round(score, 3)}
        for cust_id, score in zip(similar_customers.index, similar_customers.values)
    ]
    return recommendations

In [87]:
user_input= input("Enter CustomerID: ").strip()
print("\nTop 3 similar customers:")
if isinstance(recommendations, str):  # If the function returned an error message
    print(recommendations)
else:
    for idx, rec in enumerate(recommendations, start=1):
        print(f"{idx}. CustomerID: {rec['CustomerID']}, Similarity Score: {rec['SimilarityScore']}")

Enter CustomerID: C0001

Top 3 similar customers:
1. CustomerID: C0026, Similarity Score: 0.617
2. CustomerID: C0007, Similarity Score: 0.588
3. CustomerID: C0104, Similarity Score: 0.489


In [88]:
lookalike_recommendations = []

In [89]:
for customer_id in similarity_df.index[:20]:
    recommendations = recommend_similar_customers(customer_id, similarity_df)
    if isinstance(recommendations, list):  # If recommendations are valid
        for rec in recommendations:
            lookalike_recommendations.append({
                "InputCustomerID": customer_id,
                "LookalikeCustomerID": rec["CustomerID"],
                "SimilarityScore": rec["SimilarityScore"]
            })

In [90]:
lookalike_df = pd.DataFrame(lookalike_recommendations)

In [91]:
lookalike_df.to_csv('Kritika_Jain_Lookalike.csv', index=False)

In [92]:
print("\nThe lookalike recommendations for the first 20 customers have been saved to Kritika_Jain_Lookalike.csv.")


The lookalike recommendations for the first 20 customers have been saved to Kritika_Jain_Lookalike.csv.
